In [ ]:
from MalardClient.MalardClient import MalardClient
from MalardClient.DataSet import DataSet

import json

client = MalardClient()


In [ ]:
#Discover the parent data sets available in this environment
pds = client.getParentDataSets()

print(pds)

In [ ]:
#Query the available data sets for a given parent
datasets = client.getDataSets('cryotempo')

for ds in datasets:
    print(str(ds))


In [ ]:
#Get the bounding box for each data set
bounding_boxes = [client.boundingBox(ds) for ds in datasets]

for bb in bounding_boxes:
    print(str(bb))

In [ ]:
#get the grid cells within a bounding box
parentDs = 'cryotempo'
ds = 'GRIS_BaselineC_Q2'
region = 'greenland'

dataset = DataSet(parentDs, ds, region )
bb = client.boundingBox(dataset)

gcs = client.gridCells( dataset, bb )

#print(gcs[1])

shards = client.shards(dataset, bb )

print(shards[0].shardName)

In [ ]:
#retrieve the data for the first grid cell

resultInfo = client.executeQuery( dataset, gcs[0] )

#Print the status of the request
print( resultInfo.status )

#create a dataframe from the results
df = resultInfo.to_df

#release the underlying server resources
client.releaseCacheHandle( resultInfo.resultFileName )

In [ ]:
#executeQuery polygon. Uses the shapefile passed in to determine the extent of the bounding box.
#The projection of the shapefile needs to be consistent with the data that is passed in.
#if using lat/lon for the xCol and yCol use EPSG: 4326 and (x,y) the proj4 code from the data set
from datetime import datetime

shapeFile = "/data/puma1/scratch/mtngla/chucach/Chucach-polygon_XY.shp"

alaskaDataSet = DataSet( 'mtngla', 'ADwithTDX', 'alaska' ) 

minT = datetime(2011,1,1,0,0)
maxT = datetime(2011,2,1,0,0)

chucachDf = client.executeQueryPolygon( alaskaDataSet, minT, maxT, shapeFile=shapeFile ).to_df

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt

df = chucachDf
## plot with no filters
plt.figure(figsize=(12,10))
plt.scatter(x=df['x'], y=df['y'], c=df['elev'], marker='.', s=20, vmax=7500)
plt.colorbar()
plt.show()

In [ ]:
#get the input swath file names from the input dataframe

file_ids = df['swathFileId'].unique()

results = client.getSwathNamesFromIds( dataset, file_ids )
print(results)